# Preprocessing from Raw Dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
from src.preprocessing import apply_anomaly_detection, add_temporal_and_lag_features, remove_low_attendance_students

/tmp/ipykernel_49253/3838945402.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Load Dataset

In [2]:
df = pd.read_csv("../data/raw-dataset.csv")

In [3]:
df.head()

,id,rfid_tag,checkin_time,checkout_time,note,date
0,231966,6A3599CB,NaN,NaN,NaN,2025-12-11
1,231965,5AEC24CB,2025-12-10 21:54:41.57978+00,2025-12-11 04:12:48.000143+00,NaN,2025-12-11
2,231964,DAFD7CC0,NaN,NaN,NaN,2025-12-11
3,231963,4AECC6CB,NaN,NaN,NaN,2025-12-11
4,231962,6A48BDCB,NaN,NaN,NaN,2025-12-11


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183755 entries, 0 to 183754
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             183755 non-null  int64 
 1   rfid_tag       183755 non-null  object
 2   checkin_time   75502 non-null   object
 3   checkout_time  44666 non-null   object
 4   note           118278 non-null  object
 5   date           183755 non-null  object
dtypes: int64(1), object(5)
memory usage: 8.4+ MB


In [5]:
print(f"\nDistribusi note:")
print(df['note'].value_counts(dropna=False))


Distribusi note:
note
NaN      65477
alpa     55064
libur    52605
telat    10609
Name: count, dtype: int64


## Cleaning Outlier Student (10% rfid_tag with low attendance)

In [6]:
df_filtered = remove_low_attendance_students(df, threshold_pct=10.0)
print(f"\nDistribusi note setelah filter:")
print(df_filtered['note'].value_counts(dropna=False))

--- Removing Low Attendance Students (< 10.0%) ---
Total siswa: 1491
Siswa dengan kehadiran < 10.0%: 62
Siswa yang dipertahankan: 1429

Contoh siswa outlier yang dihapus:
rfid_tag  total_records  total_hadir  pct_hadir
1B29409D            129            5       3.88
 1B4A79D            129           12       9.30
1BD5279D            129            5       3.88
 285D2D3             30            1       3.33
2B58469D            129            0       0.00
2B604A9D            129            0       0.00
 3AC8EC3             93            7       7.53
3B7C359D            129            9       6.98
3BD0379D            129           10       7.75
 3BF049D            129            5       3.88

Data sebelum: 183755 baris
Data setelah: 176730 baris
Baris dihapus: 7025

Distribusi note setelah filter:
note
NaN      65296
libur    50598
alpa     50364
telat    10472
Name: count, dtype: int64


## Feature Engineering

In [7]:
df_preprocessed = add_temporal_and_lag_features(df_filtered)

### Without Anomaly Detection Cleaning

In [8]:
df_preprocessed.to_csv("../data/processed_data_without_anomaly.csv", index=False)

## Anomaly Detection Cleaning

In [21]:
df_clean = apply_anomaly_detection(df_preprocessed, contamination='auto')

--- Anomaly Detection (Isolation Forest) ---
Note: Applied AFTER feature engineering to preserve timeseries integrity
Features used: ['_duration_hours', '_arrival_hour', 'Count_Telat_7D', 'Count_Alpa_30D', 'Streak_Telat', 'Avg_Arrival_Time_7D']
Total data dengan checkin: 75240
Anomali terdeteksi: 16136 (21.45%)

Contoh anomali:
    rfid_tag       date                     checkin_time   note  _duration_hours  _arrival_hour
1   5AEC24CB 2025-12-11 2025-12-10 21:54:41.579780+00:00  hadir         6.301783      21.900000
8   6526ADBD 2025-12-11 2025-12-11 02:53:19.911990+00:00  telat         0.133379       2.883333
9   6AF2E8CB 2025-12-11 2025-12-10 22:40:01.779368+00:00  hadir         4.464330      22.666667
13  751A72BC 2025-12-11 2025-12-10 22:37:40.140238+00:00  hadir         5.441258      22.616667
16  A5D545BC 2025-12-11 2025-12-10 22:55:04.729709+00:00  hadir         4.407241      22.916667
18  99C7B9A6 2025-12-11 2025-12-10 22:51:30.257818+00:00  hadir         0.000000      22.85000

### With Anomaly Detection Cleaning

In [22]:
df_clean.to_csv("../data/processed_data_with_anomaly.csv", index=False)